In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
sites = ['math','stats','cstheory','dba']
urls = []
for site in sites:
    urls.append(r'https://'+site+'.stackexchange.com/questions?tab=Votes&pagesize=50')
    pass
print(urls)

['https://math.stackexchange.com/questions?tab=Votes&pagesize=50', 'https://stats.stackexchange.com/questions?tab=Votes&pagesize=50', 'https://cstheory.stackexchange.com/questions?tab=Votes&pagesize=50', 'https://dba.stackexchange.com/questions?tab=Votes&pagesize=50']


In [3]:
question_links = []

response = requests.get(urls[0])
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.find_all('a', class_='s-link')
for i in link:
    question_links.append('https://math.stackexchange.com' + i['href'])
    pass

print(question_links[1])

https://math.stackexchange.com/questions/733754/visually-stunning-math-concepts-which-are-easy-to-explain
